In [2]:
print("Hello")

Hello


# Predicting movie genres based on its poster

In [15]:
import tensorflow as tf
import numpy as np
import pandas as pd

#Utilities
import glob as gb
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data_path = "/Volumes/TSARSHAH/Datasets/Movie-Genre/movie-genre-from-its-poster/MovieGenre.csv"

##### latin-1 encoding is used to avoid utf-8 decode error, invalid byte continuation errror

In [14]:
df = pd.read_csv(data_path, encoding='latin-1')
df.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [16]:
sample_images_path = "/Volumes/TSARSHAH/Datasets/Movie-Genre/movie-genre-from-its-poster/SampleMoviePosters"

In [ ]:
img_glob = gb.glob(sample_images_path + "/" + "*.jpg")
img_dict = {}